### 1. 데이터 로드

In [2]:
# pip install nest_asyncio

In [ ]:
# pip install pymysql

In [ ]:
import os
import re
import asyncio
import openai
import pymysql
import nest_asyncio
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sklearn.metrics import precision_score, f1_score, roc_auc_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# asyncio 오류 방지 패키지(Jupyter 환경에서)
nest_asyncio.apply()

In [ ]:
# MySQL 연결 설정
db_url = os.getenv("MYSQL_URL")
engine = create_engine(db_url, pool_recycle=3600)

# MySQL 연결 테스트
try:
    with engine.connect() as conn:
        result = conn.execute(text("SELECT COUNT(*) FROM cleaned_news_data"))
        count = result.scalar()
        print(f"✅ 데이터 개수: {count} 개")
except Exception as e:
    print(f"❌ MySQL 연결 실패: {e}")

✅ 데이터 개수: 1164 개


In [ ]:
api_key = os.getenv("API_KEY")

In [14]:
# 엔진 생성
engine = create_engine(db_url, pool_recycle=3600)

### 2. 감정 분석 함수 정의

In [ ]:
# 데이터 가져오기
query = """SELECT id, content_display FROM cleaned_news_data WHERE sentiment_gpt IS NULL"""
df = pd.read_sql(query, engine)
print(f"감정 분석할 데이터 개수: {len(df)} 개")

In [ ]:
df = df.rename(columns={"content_display": "content"})  

In [ ]:
async def batch_analyze_sentiment_gpt4o(news_list, timeout=30):
    """
    GPT-4o를 이용한 감정 분석 (배치 처리).
    """
    prompt = "\n\n".join(
        [f"[뉴스 {i+1}]\n제목: {title}\n본문: {content}" for i, (title, content) in enumerate(news_list)]
    )
    prompt += """
    각 뉴스에 대해 국내 자동차 업계의 입장에서 감정을 판단하세요.

    형식:
    뉴스 1: 긍정
    뉴스 2: 부정
    뉴스 3: 긍정
    뉴스 4: 부정
    뉴스 5: 긍정

    - 오직 위의 형식만 사용하고, 다른 불필요한 문장은 포함하지 마세요.
    - 만약 감정이 애매하면 '뉴스 X: 중립'으로 답변하세요.
    - 어떤 경우에도 '판단할 수 없습니다'라는 답변은 하지 마세요.
    """

    try:
        print("🟢 GPT 요청 시작...")
        response = await asyncio.wait_for(
            client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}]
            ),
            timeout=timeout
        )
        raw_output = response.choices[0].message.content.strip()

        # ✅ GPT 응답 원본 출력
        print(f"🔵 GPT 원본 응답: {raw_output}")

        # ✅ 정규 표현식으로 감정 결과 정확하게 추출
        sentiments = []
        pattern = r"뉴스\s*(\d+)\s*:\s*(긍정|부정|중립)"
        matches = re.findall(pattern, raw_output)

        # ✅ 딕셔너리 형태로 매핑하여 순서 유지
        matches_dict = {int(num): sentiment for num, sentiment in matches}

        for i in range(1, len(news_list) + 1):
            sentiments.append(matches_dict.get(i, "N/A"))

        return sentiments

    except asyncio.TimeoutError:
        print("⚠️ GPT 응답 지연: 타임아웃 발생")
        return ["ERROR"] * len(news_list)

### 3. 감정 분석 수행

In [ ]:
# 배치 처리 실행
async def process_dataframe_batch(df, batch_size=5):
    results = []
    
    print("🔵 배치 감정 분석 시작...")

    for i in range(0, len(df), batch_size):
        print(f"🟢 배치 {i//batch_size + 1} 실행 중...")

        batch = df.iloc[i : i + batch_size]
        batch_news_list = [(str(row["id"]), row["content"]) for _, row in batch.iterrows()]
        
        try:
            sentiments = await batch_analyze_sentiment_gpt4o(batch_news_list)
            print(f"✅ 배치 {i//batch_size + 1} 응답 완료: {sentiments}")
        except Exception as e:
            print(f"❌ GPT 오류 발생: {e}")
            sentiments = ["ERROR"] * len(batch_news_list)
        
        results.extend(sentiments)
        await asyncio.sleep(0.3)

    print("🔵 모든 배치 처리 완료!")
    df["sentiment_gpt"] = results
    return df

# ✅ 기존 이벤트 루프 가져와서 실행 (Google Colab에 적합)
loop = asyncio.get_event_loop()
df = loop.run_until_complete(process_dataframe_batch(df))

# ✅ 감정 분석 결과 숫자로 변환
sentiment_mapping = {"긍정": 1, "부정": -1, "중립": 0}

df["sentiment_gpt"] = df["sentiment_gpt"].map(sentiment_mapping)
df["sentiment_gpt"] = df["sentiment_gpt"].fillna(0).astype(int)

# MySQL 업데이트
update_query = text("UPDATE cleaned_news_data SET sentiment_gpt = :sentiment_gpt WHERE id = :id")
update_data = [{"sentiment_gpt": row["sentiment_gpt"], "id": row["id"]} for _, row in df.iterrows()]

with engine.begin() as conn:
    conn.execute(update_query, update_data)

print("✅ MySQL 업데이트 완료!")

### 4. 성능 평가